In [1]:
from azureml.core import Workspace, Experiment

In [2]:
ws = Workspace.get(name="quick-starts-ws-132558")
exp = Experiment(workspace=ws, name="hyperdrive")


Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ERM9BXHBQ to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
run = exp.start_logging()

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

cluster_name = 'hd-cluster'

try:
    compute_target = ComputeTarget(ws, cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

In [19]:
ps = RandomParameterSampling(parameter_space = {'--C': choice(0.001,0.01, 0.1, 1, 10, 100, 1000), '--max_iter': choice(25, 50, 100, 150)})


policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# import azureml.core.ScriptRunConfig as srconfig
# SKLearn
est = SKLearn(source_directory = './', entry_script = 'train.py', compute_target = compute_target)

hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps, primary_metric_name='Accuracy', primary_metric_goal= PrimaryMetricGoal.MAXIMIZE, policy = policy, max_total_runs = 50, max_concurrent_runs = 8, estimator = est  )



In [20]:
experiment_1 = exp.submit(config = hyperdrive_config)

In [21]:
RunDetails(experiment_1).show()
experiment_1.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3f08acee-9596-495b-8960-b185f60063e6
Web View: https://ml.azure.com/experiments/hyperdrive/runs/HD_3f08acee-9596-495b-8960-b185f60063e6?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-132558/workspaces/quick-starts-ws-132558

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-30T23:09:32.603350][GENERATOR][INFO]Trying to sample '8' jobs from the hyperparameter space<END>\n""<START>[2020-12-30T23:09:32.063683][API][INFO]Experiment created<END>\n""<START>[2020-12-30T23:09:32.831462][GENERATOR][INFO]Successfully sampled '8' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-30T23:09:34.1256534Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END><START>[2020-12-30T23:10:05.1891314Z][SCHEDULER][INFO]The execution environment was successfully prepared.<END><START>[2020-12-30T23:10:05.1900545Z][SCHEDULER][INFO]Scheduling job, id='HD_3f08acee-9596

In [23]:
best_run = experiment_1.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

print('\n parameters:', parameter_values)


Best Run Id:  HD_3f08acee-9596-495b-8960-b185f60063e6_1

 Accuracy: 0.9128983308042489


In [24]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive,HD_3f08acee-9596-495b-8960-b185f60063e6_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Dataset
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = Dataset.Tabular.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x = clean_data(ds)

if not os.path.isdir('data'):
    os.mkdir('data')
    
# load cleaned panda dataframe x to csv file
x.to_csv("data/banking_maketing_data.csv", index=False)

#create dataset in default datastore
datastore = ws.get_default_datastore()
datastore.upload(src_dir='./data', target_path='bank-marketing', overwrite=True, show_progress=True)

 
# get data as tabular from dataset created above
data = Dataset.Tabular.from_delimited_files(path=datastore.path('bank-marketing/banking_maketing_data.csv'))

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig

automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data = data,
    label_column_name= "y",
    n_cross_validations=5,
    iterations = 12)

In [ ]:

# Submit your automl run

experiment = Experiment(ws, "automl_experiment_1")
run = experiment.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = run.get_output()

print(best_run)
print(fitted_model)

joblib.dump(value=fitted_model, filename='outputs/automl_best_model.pkl')

In [ ]:
compute_cluster.delete()

<img src='./delete.png' >